In [2]:
from lxml import html
import requests


In [3]:
s1 = [
    "https://genius.com/Game-of-thrones-winter-is-coming-annotated", 
    "https://genius.com/Game-of-thrones-the-kingsroad-annotated",
    "https://genius.com/Game-of-thrones-lord-snow-annotated",
    "https://genius.com/Game-of-thrones-cripples-bastards-and-broken-things-annotated",
    "https://genius.com/Game-of-thrones-the-wolf-and-the-lion-annotated",
    "https://genius.com/Game-of-thrones-a-golden-crown-annotated",
    "https://genius.com/Game-of-thrones-you-win-or-you-die-annotated",
    "https://genius.com/Game-of-thrones-the-pointy-end-annotated",
    "https://genius.com/Game-of-thrones-baelor-annotated",
    "https://genius.com/Game-of-thrones-fire-and-blood-annotated"
]

      


page = requests.get(url)
tree = html.fromstring(page.content)

In [8]:
import re
import pandas as pd

def process(season, episode, stripped):
    filtered = [s for s in stripped if not s.startswith("/") and s != "More on Genius"]
    labeled = []
    for f in filtered:
        m = re.search('([A-Z ]+):(.*)', f)
        if m:
            labeled.append([m.group(1),m.group(2)])
        elif f.startswith("["):
            labeled.append(["SCENE", f])
        else:
            labeled.append([f])
    merged = []
    chunks = []
    cur = "DEFAULT"
    for l in labeled:
        if len(l) >1:
            merged.append([cur,chunks])
            chunks = []
            cur = l[0]
            chunks.append(l[1])
        else:
            chunks.append(l[0])

    return [[season, episode, m[0], " ".join(m[1]).strip()] for m in merged]



In [9]:
# these were in bad shape:
#  "https://genius.com/albums/Game-of-thrones/Season-7-scripts"


seasons = [
    "https://genius.com/albums/Game-of-thrones/Season-1-scripts",
    "https://genius.com/albums/Game-of-thrones/Season-2-scripts",
    "https://genius.com/albums/Game-of-thrones/Season-3-scripts",
    "https://genius.com/albums/Game-of-thrones/Season-4-scripts",
    "https://genius.com/albums/Game-of-thrones/Season-5-scripts",
    "https://genius.com/albums/Game-of-thrones/Season-6-scripts",
]

episodes = [];
for sidx, url in enumerate(seasons):
    page = requests.get(url)
    tree = html.fromstring(page.content)
    ep_urls = tree.xpath("//div[contains(@class, 'chart_row')]/a/@href")    
    for eidx, ep in enumerate(ep_urls):
        if "needs-editing" in ep:
            print("skipping: ", ep)
            continue
        print(sidx, eidx, ep)
        page = requests.get(ep)
        tree = html.fromstring(page.content)
        raw = tree.xpath("//div[contains(@class, 'lyrics')]/descendant::*/text()")
        stripped = [r.strip() for r in raw if r.strip() != '']
        processed = process(sidx, eidx, stripped)
        episodes = episodes + processed
    

0 0 https://genius.com/Game-of-thrones-winter-is-coming-annotated
0 1 https://genius.com/Game-of-thrones-the-kingsroad-annotated
0 2 https://genius.com/Game-of-thrones-lord-snow-annotated
0 3 https://genius.com/Game-of-thrones-cripples-bastards-and-broken-things-annotated
0 4 https://genius.com/Game-of-thrones-the-wolf-and-the-lion-annotated
0 5 https://genius.com/Game-of-thrones-a-golden-crown-annotated
0 6 https://genius.com/Game-of-thrones-you-win-or-you-die-annotated
0 7 https://genius.com/Game-of-thrones-the-pointy-end-annotated
0 8 https://genius.com/Game-of-thrones-baelor-annotated
0 9 https://genius.com/Game-of-thrones-fire-and-blood-annotated
1 0 https://genius.com/Game-of-thrones-the-north-remembers-script-annotated
1 1 https://genius.com/Game-of-thrones-the-night-lands-script-annotated
1 2 https://genius.com/Game-of-thrones-what-is-dead-may-never-die-annotated
2 0 https://genius.com/Game-of-thrones-the-rains-of-castamere-annotated
skipping:  https://genius.com/Game-of-throne

In [10]:
dat = pd.DataFrame(episodes)
dat.columns = ["season_idx", "episode_idx", "character", "utterance"]
dat.to_csv("got.csv")

In [11]:
pd.options.display.max_colwidth = 1000
dat

,season_idx,episode_idx,character,utterance
0,0,0,DEFAULT,Winter is Coming Lyrics
1,0,0,SCENE,"[First scene opens with three Rangers riding through a tunnel, leaving the Wall, and going into the woods. (Eerie music in background) One Ranger splits off and finds a campsite full of mutilated bodies, including a child hanging from a tree branch. A birds-eye view shows the bodies arranged in a shield-like pattern. The Ranger rides back to the other two.]"
2,0,0,WAYMAR ROYCE,"What d’you expect? They’re savages. One lot steals a goat from another lot and before you know it, they’re ripping each other to pieces."
3,0,0,WILL,"I’ve never seen wildlings do a thing like this. I’ve never seen a thing like this, not ever in my life."
4,0,0,WAYMAR ROYCE,How close did you get?
5,0,0,WILL,Close as any man would.
6,0,0,GARED,We should head back to the wall.
7,0,0,ROYCE,Do the dead frighten you?
8,0,0,GARED,Our orders were to track the wildlings. We tracked them. They won’t trouble us no more.
9,0,0,ROYCE,You don’t think he’ll ask us how they died? Get back on your horse.
